# Processing the data (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

Мы будем обучать классификатор последовательностей на одном пакете в PyTorch следующим образом:

In [ ]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

Конечно, простое обучение модели на двух предложениях не даст очень хороших результатов. Чтобы получить лучшие результаты, вам нужно будет подготовить больший набор данных.

В этом разделе мы будем использовать в качестве примера набор данных MRPC (Microsoft Research Paraphrase Corpus), представленный в статье Уильяма Б. Долана и Криса Брокетта. Набор данных состоит из 5801 пары предложений с меткой, указывающей, являются ли они парафразами или нет (т. е. означают ли оба предложения одно и то же). Мы выбрали его для этой главы, потому что это небольшой набор данных, поэтому на нем легко экспериментировать с обучением.

<font color = "lightgreen">Hub не просто содержит модели, он также имеет несколько наборов данных на множестве разных языков. Вы можете просмотреть наборы данных здесь  
https://huggingface.co/datasets, и мы рекомендуем вам попробовать загрузить и обработать новый набор данных после того, как вы пройдете этот раздел (см. общую документацию здесь  
https://huggingface.co/docs/datasets/loading).  
Но сейчас давайте сосредоточимся на наборе данных MRPC! Это один из 10 наборов данных, составляющих бенчмарк GLUE https://gluebenchmark.com/, который является академическим бенчмарком, используемым для измерения производительности моделей машинного обучения в 10 различных задачах классификации текста.

Мы можем загрузить набор данных MRPC следующим образом:</font>

In [4]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

<font color = "lightgreen">Как вы можете видеть, мы получаем объект DatasetDict, который содержит обучающий набор, проверочный набор и тестовый набор. Каждый из них содержит несколько столбцов (sentence1, sentence2, label и idx) и переменное количество строк, которые являются количеством элементов в каждом наборе (таким образом, в обучающем наборе 3668 пар предложений, в проверочном наборе 408 и в тестовом наборе 1725).

Эта команда загружает и кэширует набор данных по умолчанию в ~/.cache/huggingface/datasets. Вспомните из Главы 2, что вы можете настроить папку кэша, установив переменную среды HF_HOME.

Мы можем получить доступ к каждой паре предложений в нашем объекте raw_datasets с помощью индексации, как в словаре:</font>

In [8]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[15]

{'sentence1': 'Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .',
 'sentence2': 'Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .',
 'label': 0,
 'idx': 16}

<font color = "lightgreen">Мы видим, что метки уже являются целыми числами, поэтому нам не придется выполнять там предварительную обработку. Чтобы узнать, какое целое число соответствует какой метке, мы можем изучить характеристики нашего raw_train_dataset. Это скажет нам тип каждого столбца:</font>

In [6]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

<font color = "lightgreen">На самом деле label имеет тип ClassLabel, а сопоставление целых чисел имени метки хранится в папке names.  
0 соответствует not_equivalent, а 1 соответствует equal.

<font color = "yellow">Для предварительной обработки набора данных нам нужно преобразовать текст в числа, которые модель может понять, это делается с помощью токенизатора. Мы можем скормить токенизатору одно предложение или список предложений, так что мы можем напрямую токенизировать все первые предложения и все вторые предложения каждой пары следующим образом:

In [9]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

<font color = "lightgreen">Как это выглядит

In [10]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

<font color = "yellow">Однако мы не можем просто передать две последовательности в модель и получить прогноз, являются ли эти два предложения парафразами или нет. Нам нужно обработать две последовательности как пару и применить соответствующую предварительную обработку. К счастью, токенизатор также может взять пару последовательностей и подготовить ее так, как ожидает наша модель BERT:

In [11]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

Итак, мы видим, что модель ожидает, что входные данные будут иметь вид [CLS] предложение1 [SEP] предложение2 [SEP], когда есть два предложения. Выравнивание этого с token_type_ids дает нам:  
'''
['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']
[      0,      0,    0,     0,       0,          0,   0,       0,      1,    1,     1,        1,     1,   1,       1]
'''

<font color = "lightgreen">Как вы можете видеть, все части ввода, соответствующие [CLS] предложение1 [SEP], имеют идентификатор типа токена 0, в то время как все другие части, соответствующие предложению2 [SEP], имеют идентификатор типа токена 1.
Обратите внимание, что если вы выберете другую контрольную точку, у вас не обязательно будут token_type_ids в ваших токенизированных входных данных (например, они не возвращаются, если вы используете модель DistilBERT). Они возвращаются только тогда, когда модель будет знать, что с ними делать, потому что она видела их во время своего предварительного обучения.
Здесь BERT предварительно обучен с идентификаторами типов токенов, и в дополнение к цели моделирования языка с маской, о которой мы говорили в Главе 1, у него есть дополнительная цель, называемая `прогнозированием следующего предложения`. Цель этой задачи — `смоделировать связь между парами предложений`.
При прогнозировании следующего предложения модели предоставляются пары предложений (со случайно замаскированными токенами) и предлагается предсказать, следует ли второе предложение за первым. Чтобы сделать задачу нетривиальной, половину времени предложения следуют друг за другом в исходном документе, из которого они были извлечены, а другую половину времени два предложения берутся из двух разных документов.
В общем, вам не нужно беспокоиться о том, есть ли token_type_id в ваших токенизированных входных данных: пока вы используете одну и ту же контрольную точку для токенизатора и модели, все будет хорошо, поскольку токенизатор знает, что предоставить своей модели.
Теперь, когда мы увидели, как наш токенизатор может работать с одной парой предложений, мы можем использовать его для токенизации всего нашего набора данных: как и в предыдущей главе, мы можем передать токенизатору список пар предложений, передав ему список первых предложений, а затем список вторых предложений. Это также совместимо с параметрами заполнения и усечения, которые мы видели в Главе 2. Итак, один из способов предварительной обработки обучающего набора данных:

In [12]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

Это работает хорошо, но у него есть недостаток в том, что возвращается словарь (с нашими ключами, input_ids, awareness_mask и token_type_ids, а также значениями, которые являются списками списков). Это также будет работать только в том случае, если у вас достаточно оперативной памяти для хранения всего набора данных во время токенизации (тогда как наборы данных из библиотеки 🤗 Datasets представляют собой файлы Apache Arrow, хранящиеся на диске, поэтому вы сохраняете только запрошенные вами образцы, загруженные в память).

Чтобы сохранить данные как набор данных, мы будем использовать метод Dataset.map(). Это также дает нам некоторую дополнительную гибкость, если нам нужно больше предварительной обработки, чем просто токенизация. Метод map() работает, применяя функцию к каждому элементу набора данных, поэтому давайте определим функцию, которая токенизирует наши входные данные:

In [13]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

<font color='yellow'>Эта функция принимает словарь (например, элементы нашего набора данных) и возвращает новый словарь с ключами input_ids, awareness_mask и token_type_ids. Обратите внимание, что это также работает, если словарь-пример содержит несколько образцов (каждый ключ как список предложений), поскольку токенизатор работает со списками пар предложений, как было показано ранее. Это позволит нам использовать опцию batched=True в нашем вызове map(), что значительно ускорит токенизацию. Токенизатор поддерживается токенизатором, написанным на Rust из библиотеки 🤗 Tokenizers. Этот токенизатор может быть очень быстрым, но только если мы дадим ему много входных данных одновременно.
Обратите внимание, что мы пока оставили аргумент padding в нашей функции токенизации. Это связано с тем, что заполнение всех образцов до максимальной длины неэффективно: лучше заполнять образцы, когда мы создаем пакет, так как тогда нам нужно заполнить только до максимальной длины в этом пакете, а не до максимальной длины во всем наборе данных. Это может сэкономить много времени и вычислительной мощности, когда входные данные имеют очень изменчивую длину!
Вот как мы применяем функцию токенизации ко всем нашим наборам данных одновременно. Мы используем batched=True в нашем вызове map, поэтому функция применяется к нескольким элементам нашего набора данных одновременно, а не к каждому элементу отдельно. Это позволяет ускорить предварительную обработку.

In [15]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

Библиотека 🤗 Datasets применяет эту обработку, добавляя новые поля в наборы данных, по одному для каждого ключа в словаре, возвращаемом функцией предварительной обработки:

In [ ]:
tokenized_datasets['train'][15]

Вы даже можете использовать многопроцессорную обработку при применении функции предварительной обработки с map(), передав аргумент num_proc. Мы не сделали этого здесь, потому что библиотека 🤗 Tokenizers уже использует несколько потоков для более быстрой токенизации наших образцов, но если вы не используете быстрый токенизатор, поддерживаемый этой библиотекой, это может ускорить вашу предварительную обработку.

Наша tokenize_function возвращает словарь с ключами input_ids, awareness_mask и token_type_ids, поэтому эти три поля добавляются во все разделы нашего набора данных. Обратите внимание, что мы также могли бы изменить существующие поля, если бы наша функция предварительной обработки возвращала новое значение для существующего ключа в наборе данных, к которому мы применили map().

Последнее, что нам нужно будет сделать, это дополнить все примеры до длины самого длинного элемента, когда мы объединяем элементы вместе — метод, который мы называем динамическим заполнением.

## Dynamic padding (динамическое заполнение)

Функция, которая отвечает за объединение образцов внутри пакета, называется функцией collate. Это аргумент, который вы можете передать при создании DataLoader, по умолчанию это функция, которая просто преобразует ваши образцы в тензоры PyTorch и объединяет их (рекурсивно, если ваши элементы являются списками, кортежами или словарями). В нашем случае это будет невозможно, поскольку входные данные, которые у нас есть, не будут иметь одинаковый размер. Мы намеренно отложили заполнение, чтобы применять его только по мере необходимости к каждому пакету и избегать слишком длинных входных данных с большим количеством заполнения. Это значительно ускорит обучение, но учтите, что если вы обучаетесь на TPU, это может вызвать проблемы — TPU предпочитают фиксированные формы, даже если для этого требуется дополнительное заполнение.

Чтобы сделать это на практике, нам нужно определить функцию collate, которая будет применять правильное количество заполнения к элементам набора данных, которые мы хотим объединить в пакет. К счастью, библиотека 🤗 Transformers предоставляет нам такую ​​функцию через DataCollatorWithPadding. Она принимает токенизатор, когда вы создаете ее экземпляр (чтобы знать, какой токен заполнения использовать и ожидает ли модель, что заполнение будет слева или справа от входов) и сделает все, что вам нужно:

In [18]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Чтобы протестировать эту новую игрушку, давайте возьмем несколько образцов из нашего обучающего набора, которые мы хотели бы объединить в пакет. Здесь мы удалим столбцы idx, sentence1 и sentence2, поскольку они не понадобятся и содержат строки (и мы не можем создавать тензоры со строками), и посмотрим на длину каждой записи в пакете:

In [19]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

Неудивительно, что мы получаем образцы разной длины, от 32 до 67. Динамическое заполнение означает, что образцы в этом пакете должны быть дополнены до длины 67, максимальной длины внутри пакета. Без динамического заполнения все образцы должны быть дополнены до максимальной длины во всем наборе данных или максимальной длины, которую может принять модель. Давайте еще раз проверим, что наш data_collator динамически дополняет пакет правильно:

In [20]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

Выглядит хорошо! Теперь, когда мы перешли от сырого текста к пакетам, с которыми может работать наша модель, мы готовы к ее тонкой настройке!

✏️ Попробуйте! Повторите предварительную обработку на наборе данных GLUE SST-2. Это немного отличается, так как состоит из отдельных предложений, а не пар, но все остальное, что мы сделали, должно выглядеть так же. Для более сложной задачи попробуйте написать функцию предварительной обработки, которая работает с любой из задач GLUE.